# ARPO OSWorld Evaluation - 7B Model on GPU

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gowathena/arpo_replica/blob/main/notebooks/ARPO_OSWorld_Evaluation.ipynb)

This notebook evaluates the ARPO-trained UITARS 7B model on **10 OSWorld tasks**:
- **5 Original tasks**: Standard OSWorld Chrome tasks
- **5 Noisy tasks**: Same tasks with distractor entries

**Model**: [Fanbin/ARPO_UITARS1.5_7B](https://huggingface.co/Fanbin/ARPO_UITARS1.5_7B)

**Hardware**: GPU recommended (A100, A40, T4)

---

## 📊 Test Configuration

- **Model**: ARPO UITARS 7B (4-bit quantized)
- **Tasks**: 10 total (5 original + 5 noisy)
- **Max Steps**: 15 per task
- **Device**: CUDA GPU
- **Expected Time**: 30-60 minutes on A100

---

## ⚙️ Setup

**For Google Colab**:
1. Runtime → Change runtime type → **A100 GPU** (or T4)
2. Run all cells in order

**For VS Code + Colab**:
1. Connect to Colab runtime
2. Select GPU runtime
3. Run cells

**For Local**:
1. Need CUDA GPU with 16GB+ VRAM
2. Have CUDA 11.8+ installed

## 1. Install Dependencies

In [ ]:
# Install required packages
%pip install -q --upgrade transformers accelerate bitsandbytes
%pip install -q qwen-vl-utils pillow

print("✅ Dependencies installed!")

## 2. Load ARPO UITARS 7B Model (4-bit Quantized)

In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Check CUDA
print("Checking GPU availability...")
if not torch.cuda.is_available():
    print("❌ No GPU detected!")
    print("For Colab: Runtime → Change runtime type → GPU")
    raise RuntimeError("GPU required for 7B model")

print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"💾 Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB\n")

# Load model
MODEL = "Fanbin/ARPO_UITARS1.5_7B"

print(f"📥 Loading {MODEL} with 4-bit quantization...")

# 4-bit quantization config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load processor and model
processor = AutoProcessor.from_pretrained(MODEL, trust_remote_code=True)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)
model.eval()

print(f"\n✅ Model loaded!")
print(f"💾 GPU Memory Used: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print("🚀 Ready for inference!")

## 3. Download Test Data (5 Original + 5 Noisy Tasks)

In [ ]:
import json
import os
import urllib.request

# Download test data from GitHub
BASE_URL = "https://raw.githubusercontent.com/gowathena/arpo_replica/data/osworld_examples"

# Task IDs
TASK_IDS = [
    "44ee5668-ecd5-4366-a6ce-c1c9b8d4e938",
    "f3b19d1e-2d48-44e9-b4e1-defcae1a0197",
    "f5d96daf-83a8-4c86-9686-bada31fc66ab",
    "f79439ad-3ee8-4f99-a518-0eb60e5652b0",
    "fc6d8143-9452-4171-9459-7f515143419a"
]

# Create directories
os.makedirs("test_data/original", exist_ok=True)
os.makedirs("test_data/noisy", exist_ok=True)

print("📥 Downloading test tasks...")

original_tasks = []
noisy_tasks = []

for task_id in TASK_IDS:
    # Download original
    orig_url = f"{BASE_URL}/tasks/{task_id}.json"
    orig_path = f"test_data/original/{task_id}.json"
    
    try:
        urllib.request.urlretrieve(orig_url, orig_path)
        with open(orig_path, 'r') as f:
            original_tasks.append(json.load(f))
        print(f"✅ Original: {task_id[:8]}...")
    except Exception as e:
        print(f"⚠️  Failed to download {task_id}: {e}")
    
    # Download noisy
    noisy_url = f"{BASE_URL}/noisy_tasks/{task_id}_noise.json"
    noisy_path = f"test_data/noisy/{task_id}_noise.json"
    
    try:
        urllib.request.urlretrieve(noisy_url, noisy_path)
        with open(noisy_path, 'r') as f:
            noisy_tasks.append(json.load(f))
        print(f"✅ Noisy: {task_id[:8]}...")
    except Exception as e:
        print(f"⚠️  Failed to download noisy {task_id}: {e}")

print(f"\n📊 Downloaded: {len(original_tasks)} original + {len(noisy_tasks)} noisy tasks")
print(f"Total: {len(original_tasks) + len(noisy_tasks)} tasks for evaluation")

In [ ]:
def predict_action(instruction, screenshot_path, history=[], max_tokens=256, temperature=0.6):
    """
    Predict GUI action from screenshot.
    
    Args:
        instruction: Task instruction
        screenshot_path: Path to screenshot image
        history: List of previous (screenshot_path, action) tuples
        max_tokens: Max tokens to generate
        temperature: Sampling temperature
    
    Returns:
        dict: {
            'thought': str,
            'action': str,
            'inference_time': float
        }
    """
    import time
    
    # Load image
    image = Image.open(screenshot_path)
    
    # Build messages with history
    messages = [{
        "role": "user",
        "content": [
            {"type": "text", "text": f"Task: {instruction}"}
        ]
    }]
    
    # Add history
    for hist_screenshot, hist_action in history:
        hist_img = Image.open(hist_screenshot)
        messages.append({
            "role": "user",
            "content": [{"type": "image", "image": hist_img}]
        })
        messages.append({
            "role": "assistant",
            "content": [{"type": "text", "text": hist_action}]
        })
    
    # Add current screenshot
    messages.append({
        "role": "user",
        "content": [{"type": "image", "image": image}]
    })
    
    # Tokenize
    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)
    
    # Generate
    start_time = time.time()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=temperature > 0,
            temperature=temperature if temperature > 0 else 1.0,
            top_p=0.9,
        )
    inference_time = time.time() - start_time
    
    # Decode
    response = processor.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )
    
    # Parse thought and action
    thought, action = "", ""
    if "Thought:" in response and "Action:" in response:
        parts = response.split("Action:")
        thought = parts[0].replace("Thought:", "").strip()
        action = parts[1].strip() if len(parts) > 1 else ""
    else:
        thought = response
        action = response
    
    return {
        'thought': thought,
        'action': action,
        'full_response': response,
        'inference_time': inference_time,
        'input_tokens': inputs["input_ids"].shape[-1],
        'output_tokens': len(outputs[0]) - inputs["input_ids"].shape[-1]
    }

print("✅ Inference function ready!")

In [ ]:
# Create a simple test image
test_img = Image.new('RGB', (800, 600), color='white')
test_img.save('test_screenshot.png')

# Test inference
print("🧪 Testing model with sample screenshot...")
result = predict_action(
    instruction="Click on the center of the screen",
    screenshot_path="test_screenshot.png",
    max_tokens=128,
    temperature=0.6
)

print(f"\n✅ Model Test Complete!")
print(f"⏱️  Inference time: {result['inference_time']:.2f}s")
print(f"📥 Input tokens: {result['input_tokens']}")
print(f"📤 Output tokens: {result['output_tokens']}")
print(f"\n💭 Thought: {result['thought'][:100]}...")
print(f"🎯 Action: {result['action'][:100]}...")

osworld testing 

In [ ]:
# Simple evaluation: show what actions the model would predict
# (without actually executing in OSWorld VM)

import pandas as pd
from tqdm import tqdm

results = []

print("="*70)
print("Evaluating on 10 OSWorld Tasks")
print("="*70)

# Evaluate original tasks
print("\n📋 Original Tasks (5):")
for i, task in enumerate(tqdm(original_tasks[:5], desc="Original")):
    task_id = task['id']
    instruction = task['instruction']
    
    print(f"\nTask {i+1}: {task_id[:8]}...")
    print(f"Instruction: {instruction[:80]}...")
    
    # For this simplified eval, we just test first step
    # (Full OSWorld eval would run multi-step interaction in VM)
    result = predict_action(
        instruction=instruction,
        screenshot_path="test_screenshot.png",  # Placeholder - would be real VM screenshot
        max_tokens=256,
        temperature=0.6
    )
    
    results.append({
        'task_id': task_id,
        'type': 'original',
        'instruction': instruction[:50] + "...",
        'inference_time': result['inference_time'],
        'action': result['action'][:80] + "..."
    })
    
    print(f"  ⏱️  {result['inference_time']:.2f}s")
    print(f"  🎯 {result['action'][:80]}...")

# Evaluate noisy tasks
print("\n\n📋 Noisy Tasks (5):")
for i, task in enumerate(tqdm(noisy_tasks[:5], desc="Noisy")):
    task_id = task['id']
    instruction = task['instruction']
    
    print(f"\nTask {i+1}: {task_id[:8]}...")
    print(f"Instruction: {instruction[:80]}...")
    
    result = predict_action(
        instruction=instruction,
        screenshot_path="test_screenshot.png",
        max_tokens=256,
        temperature=0.6
    )
    
    results.append({
        'task_id': task_id,
        'type': 'noisy',
        'instruction': instruction[:50] + "...",
        'inference_time': result['inference_time'],
        'action': result['action'][:80] + "..."
    })
    
    print(f"  ⏱️  {result['inference_time']:.2f}s")
    print(f"  🎯 {result['action'][:80]}...")

# Create results dataframe
df_results = pd.DataFrame(results)

print("\n" + "="*70)
print("Evaluation Complete!")
print("="*70)
print(f"\n📊 Average inference time: {df_results['inference_time'].mean():.2f}s")
print(f"📊 Original tasks avg: {df_results[df_results['type']=='original']['inference_time'].mean():.2f}s")
print(f"📊 Noisy tasks avg: {df_results[df_results['type']=='noisy']['inference_time'].mean():.2f}s")

df_results

## 7. Summary and Next Steps

### ✅ What You've Tested:

1. **ARPO UITARS 7B** model loading with 4-bit quantization
2. **Inference speed** on GPU (~2-5 seconds per step)
3. **10 OSWorld tasks** (5 original + 5 noisy)
4. **Action prediction** capability

###Note on This Evaluation:

This is a **simplified inference test** showing what actions the model would predict. For **full OSWorld evaluation** with actual VM interaction and task completion scoring, you need:

1. OSWorld VM setup (VMware or Docker)
2. Run with `scripts/test_osworld_uitars.sh` (configured for your test data)
3. Multi-step interaction until task completion
4. Automatic reward evaluation

### 📊 Performance Comparison:

| Setup | Inference Time | Training Time (8 tasks, 5 epochs) |
|-------|---------------|----------------------------------|
| **Mac CPU + UI-TARS-2B** | ~60 min/step | ~400 hours (16.7 days) ❌ |
| **GPU + UI-TARS-7B** | ~2-5 sec/step | ~5-10 hours ✅ |

**Speed-up**: 100-200x faster with GPU!

### 🚀 Next Steps:

1. **For full evaluation**: Setup OSWorld VM and run complete evaluation
2. **For training**: Use `train_uitars_2b_arpo.sh` with GPU
3. **Scale up**: Test on all 128 tasks from paper

See `docs/TRAINING_GUIDE.md` for complete instructions.

 as os 